# Uso efetivo da Memória do Subsistema

Agora que você pode escrever os kernels CUDA corretos e entender a importância de lançar *grids* que dão à GPU oportunidade suficiente para ocultar a latência, você aprenderá técnicas para utilizar efetivamente os subsistemas de memória da GPU. Essas técnicas são amplamente aplicáveis a uma variedade de aplicativos CUDA e algumas das mais importantes quando se trata de tornar seu código CUDA mais rápido.

Você vai começar aprendendo sobre união de memória(coalescing memory). Para desafiar sua capacidade de raciocinar sobre união de memória(coalescing memory) e expor detalhes importantes relevantes para muitos aplicativos CUDA, você aprenderá sobre *grids* bidimensionais e blocos de thread. Em seguida, você aprenderá sobre um espaço de memória sob demanda muito rápido, controlado pelo usuário, chamado memória compartilhada, e usará a memória compartilhada para facilitar a fusão de memória onde de outra forma não seria possível. Por fim, você aprenderá sobre conflitos de banco de memória compartilhada, que podem prejudicar as possibilidades de desempenho do uso de memória compartilhada e uma técnica para resolvê-los.

## Objetivos

Ao concluir esta seção, você será capaz de:
* Escrever kernels CUDA que se beneficiam de padrões de acesso à memória aglutinados.
* Trabalhar com grids multidimensionais e blocos de thread.
* Usar memória compartilhada para coordenar threads dentro de um bloco.
* Usar a memória compartilhada para facilitar os padrões de acesso à memória aglutinada.
* Resolva conflitos de banco de memória compartilhada.

## O problema: o acesso à memória não consolidada prejudica o desempenho
Antes de aprender os detalhes sobre o que é **acesso à memória combinada**, execute as células a seguir para observar as implicações de desempenho de uma alteração aparentemente trivial no padrão de acesso a dados em um kernel.

### Criação de dados
Nesta célula definimos `n` e criamos uma grid com threads iguais a `n`. Também criamos um vetor de saída com comprimento `n`. Para as entradas criamos vetores de tamanho `stride * n` por razões que serão esclarecidas abaixo:

In [ ]:
import numpy as np
from numba import cuda

n = 1024*1024 # 1M

threads_per_block = 1024
blocks = int(n / threads_per_block)

stride = 16

# Input Vectors of length stride * n
a = np.ones(stride * n).astype(np.float32)
b = a.copy().astype(np.float32)

# Output Vector
out = np.zeros(n).astype(np.float32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_out = cuda.to_device(out)


### Definição do Kernel
Em `add_experiment`, cada thread na grid adicionará um item em `a` e um item em `b` e escreverá o resultado em `out`. O kernel foi escrito de forma que podemos passar um valor `coalesced` de `True` ou `False` para afetar como ele indexa nos vetores `a` e `b`. Você verá a comparação de desempenho dos dois modos abaixo.


In [ ]:
@cuda.jit
def add_experiment(a, b, out, stride, coalesced):
    i = cuda.grid(1)
    # The above line is equivalent to
    # i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    if coalesced == True:
        out[i] = a[i] + b[i]
    else:
        out[i] = a[stride*i] + b[stride*i]

### Lançamento do Kernel Usando Acesso Agrupado (Coalesced Access)
Aqui passamos `True` como o valor `coalesced` e observamos o desempenho do kernel em várias execuções:

In [ ]:
%timeit add_experiment[blocks, threads_per_block](d_a, d_b, d_out, stride, True); cuda.synchronize()

Aqui, garantimos que o kernel funcionou conforme o esperado:

In [ ]:
result = d_out.copy_to_host()
truth = a[:n] + b[:n]
np.array_equal(result, truth)

### Resultados

>Troque o parâmetro para `False` e execute novamente. 

O desempenho do padrão de acesso a dados não agrupados(Uncoalesced) foi muito pior. Agora você aprenderá por que e como pensar sobre os padrões de acesso a dados em seus kernels para obter kernels de alto desempenho.

## Apresentação: Memória Global Aglutinada

> **Nota de rodapé**: para obter detalhes adicionais sobre o tamanho do segmento de memória global em vários dispositivos e com relação ao armazenamento em cache, consulte o [Guia de práticas recomendadas de CUDA](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#coalesced-access-to-global-memory).


In [ ]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1/coalescing-v3.pptx', 800, 450)

> _**Nota**: para detalhes adicionais sobre tamanhos de segmentos de memória global em diferentes dispositivos, e sobre memória cache, consulte [The CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#coalesced-access-to-global-memory)._

## Exercício: Somas de Colunas e Linhas

Para este exercício, você deverá escrever um kernel de somas de colunas que use padrões de acesso à memória totalmente agrupados(coalesced memory access). Para começar, você observará o desempenho de um kernel de soma de linhas que faz acessos à memória não agrupados(uncoalesced memory access).

### Somas de linha

**Criação de Dados**
Nesta célula criamos uma matriz de entrada, bem como um vetor para armazenar a solução, e transferimos cada um deles para o dispositivo. Também definimos as dimensões da grid e do bloco a serem usadas quando lançamos o kernel abaixo. Definimos uma linha arbitrária de dados para algum valor arbitrário para facilitar a verificação de correção abaixo.

In [ ]:
import numpy as np
from numba import cuda

n = 16384 # matrix side size
threads_per_block = 256
blocks = int(n / threads_per_block)

# Input Matrix
a = np.ones(n*n).reshape(n, n).astype(np.float32)
# Here we set an arbitrary row to an arbitrary value to facilitate a check for correctness below.
a[3] = 9

# Output vector
sums = np.zeros(n).astype(np.float32)

d_a = cuda.to_device(a)
d_sums = cuda.to_device(sums)

**Definição do Kernel** :
`row_sums` usará cada thread para iterar sobre uma linha de dados, somando ela e, em seguida, armazenará a soma da linha em `sums`.

In [ ]:
@cuda.jit
def row_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0
    
    for i in range(n):
        # Each thread will sum a row of `a`
        sum += a[idx][i]
        
    sums[idx] = sum

**Performance da Soma de Linha e Checagem de Corretude**

In [ ]:
%timeit row_sums[blocks, threads_per_block](d_a, d_sums, n); cuda.synchronize()

In [ ]:
result = d_sums.copy_to_host()
truth = a.sum(axis=1)
np.array_equal(truth, result)

### Soma de Colunas

**Criação de Dados**: Nesta célula criamos uma matriz de entrada, bem como um vetor para armazenar a solução, e transferimos cada um deles para o dispositivo. Também definimos as dimensões da grid e do bloco a serem usadas quando lançamos o kernel abaixo. Definimos uma coluna arbitrária de dados para algum valor arbitrário para facilitar a verificação de correção abaixo.

In [ ]:
import numpy as np
from numba import cuda

n = 16384 # matrix side size
threads_per_block = 256
blocks = int(n / threads_per_block)

a = np.ones(n*n).reshape(n, n).astype(np.float32)
# Here we set an arbitrary column to an arbitrary value to facilitate a check for correctness below.
a[:, 3] = 9
sums = np.zeros(n).astype(np.float32)

d_a = cuda.to_device(a)
d_sums = cuda.to_device(sums)


**Definição do Kernel** : `col_sums` usará cada thread para iterar sobre uma coluna de dados, somando-os e, em seguida, armazenará a soma da coluna em `sums`. Conclua a definição do kernel para fazer isso. Como exercício, faça este exemplo abaixo. 

Caso esteja com dúvidas, verifique a solução em algumas células abaixo.

In [ ]:
@cuda.jit
def col_sums(a, sums, ds):
    # TODO: Write this kernel to store the sum of each column in matrix `a` to the `sums` vector.
    pass

### **Solução**

In [ ]:
@cuda.jit
def col_sums(a, sums, ds):
  idx = cuda.grid(1)
  sum = 0.0
  
  for i in range(ds):
    # Cada thread soma uma coluna. 
    sum += a[i][idx]
      
  sums[idx] = sum

**Verifique o desempenho**: supondo que você tenha escrito `col_sums` para usar padrões de acesso agrupados, você deve ver uma velocidade significativa (quase 2x) em comparação com os `row_sums` não agrupados executados acima:

In [ ]:
%timeit col_sums[blocks, threads_per_block](d_a, d_sums, n); cuda.synchronize()

In [ ]:
result = d_sums.copy_to_host()
truth = a.sum(axis=0)
np.array_equal(truth, result)

## Blocos e grid's de 2 ou 3 dimensões

Tanto as grid's quanto os blocos podem ser configurados para conter uma coleção de blocos ou threads de 2 ou 3 dimensões, respectivamente. Isso é feito principalmente por uma questão de conveniência para programadores que geralmente trabalham com conjuntos de dados bidimensionais ou tridimensionais. Aqui está um exemplo muito trivial para destacar a sintaxe. Você pode precisar ler *ambos*, a definição do kernel e seu lançamento antes que o conceito faça sentido.

In [ ]:
import numpy as np
from numba import cuda

A = np.zeros((4,4)) # A 4x4 Matrix of 0's
d_A = cuda.to_device(A)

# Here we create a 2D grid with 4 blocks in a 2x2 structure, each with 4 threads in a 2x2 structure
# by using a Python tuple to signify grid and block dimensions.
blocks = (2, 2)
threads_per_block = (2, 2)

Este kernel irá pegar uma matriz de entrada de 0's e escrever para cada um de seus elementos, suas coordenadas (x,y) dentro da grid no formato `X.Y`:

In [ ]:
@cuda.jit
def get_2D_indices(A):
    # Pela chamada com '2' em cuda.grid, temos coordenadas unicas X e Y no grid 2D
    x, y = cuda.grid(2)
    # O codigo acima pode ser entendido como:
    # x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    # y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    
    # Escrevendo o indice x somado com o decimo de y
    A[x][y] = x + y / 10

In [ ]:
get_2D_indices[blocks, threads_per_block](d_A)
result = d_A.copy_to_host()
result

## Exercício: Adição de Matriz Bidimensional Agrupado

Nesta célula definimos `n` e criamos uma grid com threads iguais a `n`. Também criamos um vetor de saída com comprimento `n`. Para as entradas criamos vetores de tamanho `stride * n` por razões que serão esclarecidas abaixo:

In [ ]:
import numpy as np
from numba import cuda

n = 2048*2048 # 4M

# 2D blocks
threads_per_block = (32, 32)
# 2D grid
blocks = (64, 64)

# 2048x2048 input matrices
a = np.arange(n).reshape(2048,2048).astype(np.float32)
b = a.copy().astype(np.float32)

# 2048x2048 0-initialized output matrix
out = np.zeros_like(a).astype(np.float32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_out = cuda.to_device(out)

### Adicionar Matriz 2D

Seu trabalho é completar os TODOs em `matrix_add` para somar corretamente `a` e `b` em `out`. Como um desafio para sua compreensão dos padrões de acesso aglutinados, `matrix_add` aceitará um booleano `coalesced` indicando se os padrões de acesso devem ser aglutinados ou não. Ambos os modos (coalesced e uncoalesced) devem produzir resultados corretos, no entanto, você deve observar acelerações significativas abaixo ao executar com `coalesced` definido como `True`.


In [ ]:
@cuda.jit
def matrix_add(a, b, out, coalesced):
    # TODO: set x and y to index correctly such that each thread
    # accesses one element in the data.
    # x, y 
    pass
    
    if coalesced == True:
        # TODO: write the sum of one element in `a` and `b` to `out`
        # using a coalesced memory access pattern.
    else:
        # TODO: write the sum of one element in `a` and `b` to `out`
        # using an uncoalesced memory access pattern.
        

### Verifique o desempenho
Execute ambas as abaixo para iniciar `matrix_add` com os padrões de acesso combinados e não combinados que você escreveu nele e observe a diferença de desempenho. Células adicionais foram fornecidas para confirmar a exatidão do seu kernel.



**Agrupado**

In [ ]:
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out, True); cuda.synchronize()

In [ ]:
result = d_out.copy_to_host()
truth = a+b
np.array_equal(result, truth)

**Desagrupado**

In [ ]:
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out, False); cuda.synchronize()

In [ ]:
result = d_out.copy_to_host()
truth = a+b
np.array_equal(result, truth)

## Memoria compartilhada

Até agora, diferenciamos entre memória do host e memória do dispositivo, como se a memória do dispositivo fosse um único tipo de memória. Mas, na verdade, CUDA tem uma [hierarquia de memória](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#memory-hierarchy) ainda mais refinada . A memória do dispositivo que utilizamos até agora é chamada de **memória global**, que está disponível para qualquer thread ou bloco no dispositivo, pode persistir durante a vida útil do aplicativo e é um espaço de memória relativamente grande.

Discutiremos agora como utilizar uma região da memória do dispositivo no chip chamada **memória compartilhada**. A memória compartilhada é um cache definido pelo programador de tamanho limitado que [depende da GPU](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) sendo usado e é **compartilhado** entre todos os threads em um bloco. É um recurso escasso, não pode ser acessado por threads fora do bloco onde foi alocado e não persiste após o término da execução de um kernel. A memória compartilhada, no entanto, tem uma largura de banda muito maior do que a memória global e pode ser usada com grande efeito em muitos kernels, especialmente para otimizar o desempenho.

Aqui estão alguns casos de uso comuns para memória compartilhada:

 * Memória de cache lida da memória global que precisará ser lida várias vezes em um bloco.
 * Buffering da saída dos threads para que possam ser agrupados antes de serem gravados de volta na memória global.
 * Dados de preparação para operações de dispersão/reunião dentro de um bloco.



### Sintaxe de Memória Compartilhada
Numba fornece [funções](https://numba.pydata.org/numba-doc/dev/cuda/memory.html#shared-memory-and-thread-synchronization) para alocar memória compartilhada, bem como para sincronizar entre threads em um bloco, que geralmente é necessário após threads paralelas lerem ou escreverem na memória compartilhada.

Ao declarar a memória compartilhada, você fornece a forma da matriz compartilhada, bem como seu tipo, usando um [tipo Numba](https://numba.pydata.org/numba-doc/dev/reference/types.html#numba-types). **A forma da matriz deve ser um valor constante** e, portanto, você não pode usar argumentos passados para a função ou variáveis fornecidas como `numba.cuda.blockDim.x` ou os valores calculados de `numba.cuda.gridDim`. Aqui está um exemplo para demonstrar a sintaxe com comentários apontando o movimento da memória do host para a memória global do dispositivo, para a memória compartilhada, de volta para a memória global do dispositivo e, finalmente, de volta para a memória do host:

**Trocar Elementos Usando Memória Compartilhada**
O kernel a seguir recebe um vetor de entrada, onde cada thread primeiro escreverá um elemento do vetor na memória compartilhada e, depois de sincronizar de forma que todos os elementos tenham sido gravados na memória compartilhada, gravará um elemento da memória compartilhada na memória trocada vetor de saída.

Digno de nota é que cada encadeamento estará escrevendo um valor trocado da memória compartilhada que foi gravado na memória compartilhada por outro encadeamento.

Usaremos `numba.types` para definir os tipos de valores na memória compartilhada.

In [ ]:
import numpy as np
from numba import types, cuda

@cuda.jit
def swap_with_shared(vector, swapped):
    # Allocate a 4 element vector containing int32 values in shared memory.
    temp = cuda.shared.array(4, dtype=types.int32)
    
    idx = cuda.grid(1)
    
    # Move an element from global memory into shared memory
    temp[idx] = vector[idx]
    
    # cuda.syncthreads will force all threads in the block to synchronize here, which is necessary because...
    cuda.syncthreads()
    #...the following operation is reading an element written to shared memory by another thread.
    
    # Move an element from shared memory back into global memory
    swapped[idx] = temp[3 - cuda.threadIdx.x] # swap elements


**Criação dos Dados**

In [ ]:
vector = np.arange(4).astype(np.int32)
swapped = np.zeros_like(vector)

# Move host memory to device (global) memory
d_vector = cuda.to_device(vector)
d_swapped = cuda.to_device(swapped)
vector

In [ ]:
swap_with_shared[1, 4](d_vector, d_swapped)

**Checagem de Resultados**

In [ ]:
  # Move device (global) memory back to the host
result = d_swapped.copy_to_host()
result

## Apresentação: Memória Compartilhada para Memória Aglutinada 

In [ ]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1/shared_coalescing.pptx', 800, 450)

## Exercício: memória compartilhada usada para leituras e gravações combinadas com transposição de matriz
Neste exercício, você implementará o que acabou de ser demonstrado na apresentação, escrevendo um kernel de transposição de matriz que, usando memória compartilhada, faz leituras e gravações aglutinadas na matriz de saída na memória global.

### Leituras combinadas, gravações não combinadas
Como referência e para comparação de desempenho, aqui está um kernel de transposição de matriz ingênua que faz leituras agrupadas da entrada, mas gravações não agrupadas na saída.

**Criação de Dados**
Aqui criamos uma matriz de entrada 4096x4096 `a`, bem como uma matriz de saída 4096x4096 `transposed` e copiamos para o dispositivo.

Também definimos um grid bidimensional com blocos bidimensionais a serem usados abaixo. Observe que criamos um grid com um número total de threads igual ao número de elementos na matriz de entrada.

In [ ]:
from numba import cuda
import numpy as np

n = 4096*4096 # 16M

# 2D blocks
threads_per_block = (32, 32)
#2D grid
blocks = (128, 128)

# 4096x4096 input and output matrices
a = np.arange(n).reshape((4096,4096)).astype(np.float32)
transposed = np.zeros_like(a).astype(np.float32)

d_a = cuda.to_device(a)
d_transposed = cuda.to_device(transposed)

**Núcleo de transposição de matriz ingênua**
Este kernel transpõe `a` corretamente, escrevendo a transposição para `transposed`. Ele faz leituras de `a` de maneira combinada, no entanto, suas gravações em `transposed` não são combinadas.

In [ ]:
@cuda.jit
def transpose(a, transposed):
    x, y = cuda.grid(2)

    transposed[x][y] = a[y][x]

**Chegagem de Performance e Corretude**

In [ ]:
%timeit transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()

In [ ]:
result = d_transposed.copy_to_host()
expected = a.T
np.array_equal(result, expected)

### Refatoração para leituras e gravações combinadas
Seu trabalho será refatorar o kernel `transpose` para usar a memória compartilhada e fazer leituras e gravações da memória global de maneira combinada.

In [ ]:
import numpy as np
from numba import cuda, types as numba_types


n = 4096*4096 # 16M

# 2D blocks
threads_per_block = (32, 32)
#2D grid
blocks = (128, 128)

# 4096x4096 input and output matrices
a = np.arange(n).reshape((4096,4096)).astype(np.float32)
transposed = np.zeros_like(a).astype(np.float32)

d_a = cuda.to_device(a)
d_transposed = cuda.to_device(transposed)

**Exercício: Escreva um Kernel de Transposição que Use Memória Compartilhada**

Complete os TODOs dentro da definição do kernel `tile_transpose`.

In [ ]:
@cuda.jit
def tile_transpose(a, transposed):
    # `tile_transpose` é o kernel enviado, assumimos que é lançado como um bloco de dimensao 32 x 32,
    #e 'a' é um multiplo dessas dimensões.
    
    # 1) Crie um bloco 32x32 de memoria compartilhada.
    # TODO: Seu Código Aqui.
    


    # Compute o "offset entre a global e a memo. compartilhada. 
    # Use o acesso aglutinado/compartilhado que queremos mapear por meio do incremento de threadIdx.x.
    # dica: a melhor mudança é mudando o indice dos dados, que nada é a ordem (x, ou y) da nossa matriz 
    # Nota: `a_col` and `a_row` ja estao corretos!
    
    a_col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    a_row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y


    # 2) Faça a leitura de agrupamento, isto é, a escrita da global para a memória temporaria
    # dica: use os indices de thread !
    # TODO: Seu Código Aqui.
    
    
    # 3) Espere que todas as threads terminem, para seguir para o próx. passo
    # TODO: Seu Código Aqui.



    # 4) Calcule o local da transposição para a memoria compartilhada (tile)
    # para ser alocada na posicao correta da memoria global. Note que blockIdx.y*blockDim.y
    # e blockIdx.x * blockDim.x estão trocadas (porque queremos escrever na posição transposta)
    # mas queremos manter o acesso aglutinado, e associar threadIdx.x para a mudança de indice
    # mais rapida possivel, de maneira similar com as colunas.
    # Nota: `t_col` e `t_row` ja estao corretos !

    t_col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.x
    t_row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.y

    # 5) Escreva da memória compartilhada (usando os  indices da thread)
    # de volta para a memória global (usando indices do grid)
    # transpondo cada elemento dentro da memória compartilhada.
    # TODO: Seu Código Aqui.
    
    
    

**Checagem de Performance e Corretude**
Verifique o desempenho do seu kernel de transposição refatorado. Você deve ver uma aceleração em comparação com o desempenho de transposição da linha de base acima.

In [ ]:
%timeit tile_transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()

In [ ]:
result = d_transposed.copy_to_host()
expected = a.T
np.array_equal(result, expected)

### Por que uma melhoria tão pequena?

Embora isso seja uma aceleração significativa para apenas algumas linhas de código, você pode pensar que a melhoria de desempenho não é tão forte quanto o esperado com base em melhorias de desempenho anteriores para usar padrões de acesso combinados. Existem 2 razões principais para isso:

1. O kernel transpose ingênuo estava fazendo leituras combinadas, portanto, sua versão refatorada otimizou apenas metade do acesso à memória global durante a execução do kernel.
2. Seu código como escrito sofre de algo chamado conflitos de banco de memória compartilhada, um tópico para o qual voltaremos nossa atenção agora.

## Apresentação: Memória Compartilhada para Memória Aglutinada

In [ ]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1/bank_conflicts.pptx', 800, 450)

## Avaliação: resolver conflitos de banco de memória

Como exercício final, você refatorará o kernel de transposição utilizando memória compartilhada para ser livre de conflitos de banco de memória compartilhada.

In [ ]:
import numpy as np
from numba import cuda, types

n = 4096*4096 # 16M
threads_per_block = (32, 32)
blocks = (128, 128)

a = np.arange(n).reshape((4096,4096)).astype(np.float32)
transposed = np.zeros_like(a).astype(np.float32)

d_a = cuda.to_device(a)
d_transposed = cuda.to_device(transposed)

### Torne o Banco do Kernel Livre de Conflitos

O kernel `tile_transpose_conflict_free` é um kernel de transposição de matriz funcional que utiliza memória compartilhada para que leituras e gravações na memória global sejam combinadas. Seu trabalho é refatorar o kernel para que ele não sofra conflitos de banco de memória.


In [ ]:
@cuda.jit
def tile_transpose_conflict_free(a, transposed):
    
    # `tile_transpose` é o kernel enviado, assumimos que é lançado como um bloco de dimensao 32 x 32,
    #e 'a' é um multiplo dessas dimensões.
    
    # 1) Crie um bloco 32x32 de memoria compartilhada.
    tile = cuda.shared.array((32, 32), numba_types.float32)

    # Compute o "offset entre a global e a memo. compartilhada. 
    x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    
    # 2) Faça a leitura de agrupamento, isto é, a escrita da global para a memória temporaria
    # Note the use of local thread indices for the shared memory write,
    # and global offsets for global memory read.
    tile[cuda.threadIdx.y, cuda.threadIdx.x] = a[y, x]

    # 3) Espere que todas as threads terminem, para seguir para o próx. passo
    cuda.syncthreads()
    
    # 4) Calcule o local da transposição para a memoria compartilhada (tile)
    # para ser alocada na posicao correta da memoria global. 
    t_x = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.x
    t_y = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.y

    # 5) Escreva da memória compartilhada (usando os  indices da thread)
    # de volta para a memória global (usando indices do grid)
    transposed[t_y, t_x] = tile[cuda.threadIdx.x, cuda.threadIdx.y]


### Chegagem de Performance e Corretude
Supondo que você tenha resolvido corretamente os conflitos de banco, esse kernel deve executar significativamente mais rápido que o kernel de transposição ingênuo e o kernel de transposição de memória compartilhada (com conflitos de banco). Para passar na avaliação, seu kernel precisará rodar em média em menos de 840 µs.

O primeiro valor impresso ao executar a célula a seguir fornecerá o tempo médio de execução do seu kernel.


In [ ]:
%timeit tile_transpose_conflict_free[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()

In [ ]:
result = d_transposed.copy_to_host()
expected = a.T
np.array_equal(result, expected)

>Seu código deve executar em menos de 840µs!